In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Simulate 60 days of data
customers = ['Amazon', 'Walmart', 'Target', 'Costco', 'BestBuy', 'eBay']
data = []
start_date = datetime(2025, 4, 1)
for i in range(1, 61):
    date = start_date + timedelta(days=i)
    for _ in range(random.randint(3, 6)):
        data.append({
            'id': random.randint(1000, 9999),
            'customer': random.choice(customers),
            'date': date.date().isoformat(),
            'amount': random.randint(100, 2000),
            'last_updated': (date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59))).isoformat()
        })

df = pd.DataFrame(data)
df.to_csv('sales_data_large.csv', index=False)
df.head()


,id,customer,date,amount,last_updated
0,3636,Walmart,2025-04-02,1386,2025-04-02T22:55:00
1,2615,Walmart,2025-04-02,925,2025-04-02T01:48:00
2,6180,Amazon,2025-04-02,1950,2025-04-02T01:35:00
3,7060,BestBuy,2025-04-02,355,2025-04-02T13:22:00
4,8236,Costco,2025-04-02,581,2025-04-02T10:29:00


In [7]:
# FULL EXTRACTION
df_full = pd.read_csv("sales_data_large.csv", parse_dates=["last_updated"])
print(f"Extracted {len(df_full)} rows fully.")
df_full.head()


Extracted 275 rows fully.


,id,customer,date,amount,last_updated
0,3636,Walmart,2025-04-02,1386,2025-04-02 22:55:00
1,2615,Walmart,2025-04-02,925,2025-04-02 01:48:00
2,6180,Amazon,2025-04-02,1950,2025-04-02 01:35:00
3,7060,BestBuy,2025-04-02,355,2025-04-02 13:22:00
4,8236,Costco,2025-04-02,581,2025-04-02 10:29:00


In [ ]:
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-20 12:00:00")

# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

df = pd.read_csv("sales_data_large.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)

df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Extracted {len(df_incremental)} rows incrementally since {last_extraction}.")
df_incremental.head()


Extracted 192 rows incrementally since 2025-04-20 12:00:00.


,id,customer,date,amount,last_updated
82,8949,Costco,2025-04-20,1580,2025-04-20 23:12:00
84,6858,Amazon,2025-04-20,632,2025-04-20 16:32:00
85,2038,Amazon,2025-04-21,1317,2025-04-21 03:20:00
86,7091,Amazon,2025-04-21,954,2025-04-21 15:47:00
87,7139,Walmart,2025-04-21,1090,2025-04-21 02:54:00


In [10]:
# Update checkpoint
new_checkpoint = df['last_updated'].max()
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")


Updated last_extraction.txt to 2025-05-31 23:12:00


In [11]:
# SECTION 4: Transform Full Data

# Remove duplicates
df_full = df_full.drop_duplicates()

# Fill missing values
df_full["customer"].fillna("Unknown", inplace=True)
df_full["amount"].fillna(0, inplace=True)

# Standardize date format
df_full["date"] = pd.to_datetime(df_full["date"], errors='coerce')

# Add amount_with_tax column (16% tax)
df_full["amount_with_tax"] = (df_full["amount"] * 1.16).round(2)

# Categorize amount values
df_full["amount_category"] = pd.cut(
    df_full["amount"],
    bins=[0, 500, 1000, 1500, float("inf")],
    labels=["Low", "Medium", "High", "Very High"]
)

# Save transformed data
df_full.to_csv("transformed_full.csv", index=False)
df_full.head()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10332\1868575224.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full["customer"].fillna("Unknown", inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10332\1868575224.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

,id,customer,date,amount,last_updated,amount_with_tax,amount_category
0,3636,Walmart,2025-04-02,1386,2025-04-02 22:55:00,1607.76,High
1,2615,Walmart,2025-04-02,925,2025-04-02 01:48:00,1073.00,Medium
2,6180,Amazon,2025-04-02,1950,2025-04-02 01:35:00,2262.00,Very High
3,7060,BestBuy,2025-04-02,355,2025-04-02 13:22:00,411.80,Low
4,8236,Costco,2025-04-02,581,2025-04-02 10:29:00,673.96,Medium


In [12]:
# SECTION 5: Transform Incremental Data

# Remove duplicates
df_incremental = df_incremental.drop_duplicates()

# Fill missing values
df_incremental["customer"].fillna("Unknown", inplace=True)
df_incremental["amount"].fillna(0, inplace=True)

# Standardize date format
df_incremental["date"] = pd.to_datetime(df_incremental["date"], errors='coerce')

# Add amount_with_tax column (16% tax)
df_incremental["amount_with_tax"] = (df_incremental["amount"] * 1.16).round(2)

# Categorize amount values
df_incremental["amount_category"] = pd.cut(
    df_incremental["amount"],
    bins=[0, 500, 1000, 1500, float("inf")],
    labels=["Low", "Medium", "High", "Very High"]
)

# Save transformed data
df_incremental.to_csv("transformed_incremental.csv", index=False)
df_incremental.head()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10332\2714324874.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_incremental["customer"].fillna("Unknown", inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10332\2714324874.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

,id,customer,date,amount,last_updated,amount_with_tax,amount_category
82,8949,Costco,2025-04-20,1580,2025-04-20 23:12:00,1832.80,Very High
84,6858,Amazon,2025-04-20,632,2025-04-20 16:32:00,733.12,Medium
85,2038,Amazon,2025-04-21,1317,2025-04-21 03:20:00,1527.72,High
86,7091,Amazon,2025-04-21,954,2025-04-21 15:47:00,1106.64,Medium
87,7139,Walmart,2025-04-21,1090,2025-04-21 02:54:00,1264.40,High
